In [10]:
from owlready2 import *
from owlready2.pymedtermino2 import *
from owlready2.pymedtermino2.umls import *
# from owlready2.pymedtermino2.icd10_french import *

from tqdm import tqdm
import pandas as pd

import random

ModuleNotFoundError: No module named 'pandas'

In [11]:
! pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 45.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 64.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 kB 9.3 MB/s eta 0:00:0000:01


In [8]:
DB_NAME = "../../../data/pym.sqlite3"
ZIP_NAME = "../../../data/umls-2024AA-full.zip"
SAVE_TO = "../../../data/triplets.tsv"


BANNED_PROPS = ['icd-o-3_code', 'term_type', 'subset_member', 'label', 'terminology', 'synonyms', 'unifieds']
RANDOM_SEED=30_239_566

In [3]:
default_world.set_backend(filename = DB_NAME)
PYM = get_ontology("http://PYM/").load()

# Extracting concept

In [6]:
def get_concept_triplets(concept, seed=None):
    triplets = []
    
    # Set random seed for reproducibility if provided
    if seed is not None:
        random.seed(seed)
    
    # Iterate through all properties of the concept
    for prop in concept.get_class_properties():
        # Skip the banned properties
        if prop.name in BANNED_PROPS:
            continue
        
        # Get the linked concept(s) via the property
        related_concepts = getattr(concept, prop.name)
        
        # If related concepts are a list or set, select one randomly
        if isinstance(related_concepts, list) or isinstance(related_concepts, set):
            if related_concepts:
                selected_concept = random.choice(list(related_concepts))
                if hasattr(selected_concept, 'label') and selected_concept.label:
                    triplets.append((str(concept.label[0]), prop.name, str(selected_concept.label[0])))
                else:
                    triplets.append((str(concept.label[0]), prop.name, str(selected_concept)))
        else:
            # If it's a single concept
            if hasattr(related_concepts, 'label') and related_concepts.label:
                triplets.append((str(concept.label[0]), prop.name, str(related_concepts.label[0])))
            else:
                triplets.append((str(concept.label[0]), prop.name, str(related_concepts)))
    
    return triplets

# Function to get triplets for all concepts in the ontology
def get_all_concept_triplets(ontology, seed=None):
    all_triplets = []

    # Iterate over all concepts (classes) in the ontology
    for concept in tqdm(ontology.classes()):
        try:
            triplets = get_concept_triplets(concept, seed)  # Use the function from before
            all_triplets.extend(triplets)  # Add the triplets to the full list
        except IndexError:
            pass
    return all_triplets

In [7]:
all_triplets = get_all_concept_triplets(PYM, seed=RANDOM_SEED)
# for triplet in all_triplets:
#     print(triplet)

1220039it [07:06, 2863.03it/s]


## Save it

In [9]:
df = pd.DataFrame(all_triplets, columns=['Concept', 'Property', 'Related Concept'])
# Save the DataFrame to a CSV file
df.to_csv(SAVE_TO, index=False, sep='\t')

NameError: name 'pd' is not defined